In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-02"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
4189,2024-06-02,Israel Superliga,12:00,Maccabi Ironi Ramat Gan,Ironi Kiryat Ata,1.37,3.06,168.5,1.89,1.89,0.0,0.00,0.00,Cbf9Lgp3,0.729927,0.326797,0.529101,0.529101,0.056724,199.030,93.075689,0.467647,0.0,0.000000,NaN,137.76,2.580,1.351074,0.523672,-28.0,309.382,227.559607,0.735530,1.2,1.643168,1.369306,211.50,3.564,2.220626,0.623071,-31.0,84,94,1.64,2.25,164.528,329.640,0.539508,0.000000,NaN,-2.12,-0.424,-0.872642,0.596723,0.6,0.003277,-5.00,-1.000,-2.060000,0.341024,0.2,-0.141024
4190,2024-06-02,Polônia Liga De Basquete,15:50,Trefl Sopot,Szczecin,1.61,2.25,163.5,1.86,1.80,0.0,0.00,0.00,Wn5oaRll,0.621118,0.444444,0.537634,0.555556,0.065562,134.488,39.083659,0.290611,1.8,1.643168,0.912871,118.26,1.612,0.253417,0.157206,15.0,166.340,54.985756,0.330562,3.0,0.000000,0.000000,182.86,1.838,0.529783,0.288239,61.0,73,82,1.62,2.23,126.004,211.888,0.234481,0.023184,NaN,-0.88,-0.176,-3.465909,0.753173,0.7,-0.053173,3.79,0.758,1.649077,0.502695,0.9,0.397305
4191,2024-06-02,Israel Superliga,13:00,Maccabi Tel Aviv,Nes Ziona,1.06,9.00,172.5,1.86,1.80,0.0,0.00,0.00,K47Ncjhd,0.943396,0.111111,0.537634,0.555556,0.054507,122.352,9.576363,0.078269,2.4,1.341641,0.559017,118.80,1.344,0.214779,0.159806,58.0,183.114,77.344095,0.422382,2.4,1.341641,0.559017,297.66,2.328,1.293723,0.555723,22.0,99,66,1.20,4.51,112.934,258.178,1.116188,0.023184,NaN,1.02,0.204,0.294118,0.865731,1.0,0.134269,-0.80,-0.160,-50.000000,0.424576,0.3,-0.124576
4192,2024-06-02,Israel Superliga,14:40,Hapoel Jerusalem,Hapoel Holon,1.31,3.41,158.5,1.87,1.79,0.0,0.00,0.00,0Y5PHBqS,0.763359,0.293255,0.534759,0.558659,0.056614,148.002,18.300405,0.123650,3.0,0.000000,0.000000,150.50,1.772,0.322676,0.182097,62.0,163.726,46.110824,0.281634,1.2,1.643168,1.369306,114.92,2.244,0.814297,0.362878,-34.0,70,68,2.15,1.69,127.600,163.278,0.629205,0.030912,NaN,-0.31,-0.062,-5.000000,0.738796,0.7,-0.038796,-1.33,-0.266,-9.060150,0.493233,0.4,-0.093233
4193,2024-06-02,Israel Superliga,15:05,Hapoel Tel-Aviv,H. Haifa,1.07,8.40,169.5,1.90,1.88,0.0,0.00,0.00,8QmIJXEF,0.934579,0.119048,0.526316,0.531915,0.053627,127.512,39.684875,0.311225,1.8,1.643168,0.912871,117.00,1.560,0.518507,0.332377,23.0,208.578,136.733201,0.655549,1.2,1.643168,1.369306,450.45,2.512,1.417893,0.564448,-40.0,100,91,1.17,4.95,100.994,191.504,1.094634,0.007483,NaN,-1.31,-0.262,-0.267176,0.794599,0.8,0.005401,1.89,0.378,19.576720,0.380379,0.6,0.219621
4194,2024-06-02,Itália Série A2,13:00,Agrigento,Luiss Roma,1.42,2.72,160.5,1.84,1.86,0.0,0.00,0.00,bLwGzOKt,0.704225,0.367647,0.543478,0.537634,0.071872,157.236,38.997035,0.248016,1.2,1.643168,1.369306,221.00,2.138,0.705103,0.329796,-25.0,200.808,71.615029,0.356634,0.6,1.341641,2.236068,209.44,2.602,0.771764,0.296604,-24.0,68,77,3.25,2.72,166.174,264.154,0.444077,0.007644,NaN,1.44,0.288,1.458333,0.463649,0.4,-0.063649,-5.00,-1.000,-1.720000,0.268867,0.1,-0.168867
4195,2024-06-02,Itália Série A2,13:00,Chiusi,Cento,1.36,2.97,146.5,1.86,1.84,0.0,0.00,0.00,pxxOY1kg,0.735294,0.336700,0.537634,0.543478,0.071994,112.340,22.170121,0.197348,1.8,1.643168,0.912871,73.81,1.632,0.549973,0.336993,12.0,112.346,17.814634,0.158569,2.4,1.341641,0.559017,103.49,1.428,0.151228,0.105902,31.0,61,79,1.21,1.31,150.494,179.868,0.525839,0.007644,NaN,1.46,0.292,1.232877,0.509359,0.6,0.090641,-0.62,-0.124,-1

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
4190,15:50,Polônia Liga De Basquete,Trefl Sopot,Szczecin,1.61,Back Home
4198,03:30,Nova Zelândia Nbl,Manawatu Jets,Otago Nuggets,2.62,Back Home
4208,19:00,Chile Lnb 2,Municipal Chillan,Sergio Ceppi,2.56,Back Home
